# Unet based 1-D CNN segmentation training

## Description

This notebook demonstrates the process of training a 1-D CNN segmentation model using a U-Net architecture. In it we perform data preprocessing, model definition, training, validation, and evaluation.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib
import segmentation_models_pytorch as smp
import random
random.seed(10)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

#The notebook has not been tested on GPUs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: ", device)

In [ ]:
from src.Unet_1D_model import build_unet
from src.data_augmentation import augment_data

The data is split into train, validation and testing sets with a split of 70%,20%,10%.

In [ ]:
data_path = pathlib.Path("data")
with open(data_path / "labeled_data_fileNames.txt") as f:
    _list_of_files = f.readlines()
list_of_files = [x.replace('\n','') for x in _list_of_files]
print(list_of_files)
print(len(list_of_files))

all_data=[]
all_labels=[]

for n,file in enumerate(list_of_files):
    data,labels=np.loadtxt(data_path / file,delimiter=',')

    rsc=RobustScaler(quantile_range=(30.0, 70.0),unit_variance=True)
    data=data.reshape(-1,1)
    
    data=rsc.fit_transform(data)

    all_data.append(data)
    all_labels.append(labels)
X=np.array(all_data)
y=np.array(all_labels)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=101)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.22, random_state=101) # 0.22 x 0.9 = 0.2



## Windowing

To be able to train a CNN on the input data, it needs to be split into windows of equal size. The window size was set to 2000, with a stride of 1000 so that if a peak is split in half by the window another window would have the full peak in the middle. The last window consists of the last data points of the series as well as looped data from the start so a 2000 size window could be created.



In [ ]:
import numpy as np

window_size=2000
stride=window_size//2
# print(stride)

def create_windows(data, window_size, stride):
    missing_vals = window_size - (len(data) % window_size)
    data_looped = np.append(data, data[:missing_vals + window_size])
    return np.array([data_looped[i:i + window_size] for i in range(0, len(data) + 1, stride)])


def windowing(data,labels):
    data_windows = []
    label_windows = []
    for data, label in zip(data, labels):
        data_windows.append(create_windows(data, window_size, stride))
        label_windows.append(create_windows(label, window_size, stride))
    return np.concatenate(data_windows,dtype=np.float32), np.concatenate(label_windows)

X_train_windows, y_train_windows = windowing(X_train, y_train)
X_val_windows, y_val_windows = windowing(X_val, y_val)
X_test_windows, y_test_windows = windowing(X_test, y_test)




## Data preparation

Here we prepare the windowed data for training in a pytorch model. We onehot encode the target values. Then we put the training, validation and test datasets into a TimeSeriesDataSet structure. One can also add augmentation to the training set, in theory this will help with generalization but it didn't give much effect in this case.

In [ ]:
n_features = 1

X_train_input = X_train_windows.reshape((len(X_train_windows), n_features, window_size))
X_val_input = X_val_windows.reshape((len(X_val_windows), n_features, window_size))
X_test_input = X_test_windows.reshape((len(X_test_windows), n_features, window_size))

train_target_onehot=torch.nn.functional.one_hot(torch.tensor(y_train_windows).long(),num_classes=2)
train_target_onehot=train_target_onehot.transpose(1,2)
val_target_onehot=torch.nn.functional.one_hot(torch.tensor(y_val_windows).long(),num_classes=2)
val_target_onehot=val_target_onehot.transpose(1,2)
test_Y_windows_onehot=torch.nn.functional.one_hot(torch.tensor(y_test_windows).long(),num_classes=2)
test_Y_windows_onehot=test_Y_windows_onehot.transpose(1,2)


In [ ]:
from torch.utils.data import Dataset, ConcatDataset
batch_size = 16

class TimeSeriesDataset(Dataset):
    def __init__(self, data, targets, augment=False):
        print(data.shape)
        if augment:
            data = augment_data(data)
        print(data.shape)
        self.data = torch.from_numpy(data.astype('float32').copy()).clone().to(device)
        self.targets = targets.clone().to(device)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.targets[idx]
        
        return x,y

# Create original dataset
train_dataset = TimeSeriesDataset(X_train_input, train_target_onehot, augment=False)

# Create augmented dataset
train_dataset_augmented = TimeSeriesDataset(X_train_input, train_target_onehot, augment=True)

# Combine original and augmented datasets
train_dataset = ConcatDataset([train_dataset, train_dataset_augmented])
# Create DataLoader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


val_dataset = torch.utils.data.TensorDataset(torch.tensor(X_val_input).to(device), val_target_onehot)
test_dataset=torch.utils.data.TensorDataset(torch.tensor(X_test_input).to(device), test_Y_windows_onehot)

valid_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Model

The model used is a 1-D version of the UNet segmentation model. Below is code used to print a summary of the model and it's parameters. The proper model used during training is defined lower down.

In [ ]:
from torchsummary import summary
model = build_unet(input_channels=1, num_classes=2,layer_n=24,conv_kernel_size=3,scaling_kernel_size=2)
print(summary(model, (1, window_size), batch_size=batch_size, device=device.type))

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
import time
from sklearn.metrics import f1_score,precision_score,recall_score

## Training

Here we import the training loop from src/CNN_training.py and perform training with the parameters we choose. The model class is defined in src/Unet_1D_model.py. The arguments to the training loop are:

- data_loaders (tuple): A tuple containing the training and validation data loaders.
- config (dict): A dictionary containing the configuration parameters for the model and training process.

The expected keys in the config which include many hyperparameters of the model:

- 'num_classes' (int): Number of output classes.
- 'layer_n' (int): Number of layers in the U-Net model.
- 'dropout' (float): Dropout rate.
- 'loss_fn' (str): Loss function to use ('CE','BCE', 'DL', 'DLLog').
- 'lr' (float): Learning rate.
- 'batch_size' (int): Batch size.
- 'n_epochs' (int): Number of epochs.
- 'saveString' (str): Prefix for saving model checkpoints.

In [ ]:
from src.CNN_training import training_loop

config={'layer_n':24,
        'dropout':0.2,
        'batch_size':batch_size,
        'n_epochs':30,
        'lr':0.0003,
        'saveString':'generatedData_RobustScaled3070_1DUnet',
        'loss_fn':'BCE',
        'save_best_training_model':False,
        'num_classes':2,}

model,loss_values, version_name = training_loop((train_loader,valid_loader), config)

# loss_values=np.array(loss_values)
plt.figure()
plt.plot(loss_values[:,0],loss_values[:,1],label='train_loss')
plt.plot(loss_values[:,0],loss_values[:,5],label='val_loss')
plt.xlabel("epochs")
plt.legend()
plt.savefig("figures/"+version_name+"_loss.png")
# plt.show()

plt.figure()
plt.plot(loss_values[:,0],loss_values[:,2],label='train_f1')
plt.plot(loss_values[:,0],loss_values[:,6],label='val_f1')
plt.xlabel("epochs")
plt.legend()
plt.savefig("figures/"+version_name+"_f1.png")
# plt.show()



In [ ]:
plt.figure()
plt.plot(loss_values[:,0],loss_values[:,1],label='train_loss')
plt.plot(loss_values[:,0],loss_values[:,5],label='val_loss')
plt.xlabel("epochs")
plt.legend()
# plt.savefig("figures/"+version_name+"_loss.png")
plt.show()

plt.figure()
plt.plot(loss_values[:,0],loss_values[:,2],label='train_f1')
plt.plot(loss_values[:,0],loss_values[:,6],label='val_f1')
plt.xlabel("epochs")
plt.legend()
# plt.savefig("figures/"+version_name+"_f1.png")
plt.show()

## Testing

Here we test the performance of the model on the test dataset. The model can be loaded from the checkpoint. We can also show some plots of the windows to see the performance of the network.

In [ ]:
# Version name can be defined manually or taken from the training loop
# version_name="T015_event2_event9_widelabeled_RobustScaled3070_noBN_build_unet_layer24_dropout0.2_conv3_scaling2_lossBCE_lr0.0005_bs16_epochs50"

model = torch.jit.load("Models/" + version_name + "_validationcheckpoint_Models.pt")

# model = torch.jit.load('fullModel/T015_event2_event9_widelabeled_RobustScaled3070_noBN_build_unet_layer20_dropout0.0_conv3_scaling2_lossBCE_lr0.001_bs32_validationcheckpoint_fullmodel.pt')

model.eval()

test_shape = (len(test_loader.dataset),) + test_loader.dataset[0][0].shape
test_preds = np.zeros((int(test_shape[0] * test_shape[2])))
test_targets = np.zeros((int(test_shape[0] * test_shape[2])))
import sys

np.set_printoptions(threshold=sys.maxsize)


total = test_targets.size

test_shape = (len(test_loader.dataset),) + test_loader.dataset[0][0].shape

for i, (x_batch, y_batch) in enumerate(test_loader):
    y_pred = model(x_batch).detach()

    pred = F.softmax(y_pred, 1).detach().cpu().numpy().argmax(axis=1)
    y_batch_pred = y_batch.detach().cpu().numpy().argmax(axis=1)

    test_preds[
        i * batch_size * test_shape[2] : (i + 1) * batch_size * test_shape[2]
    ] = pred.reshape((-1))
    test_targets[
        i * batch_size * test_shape[2] : (i + 1) * batch_size * test_shape[2]
    ] = y_batch_pred.reshape((-1))

    # Plot some windows to see the predictions
    if i < 5:
        j = 4
        print(j * 1000)
        x_batch0 = x_batch[j].flatten().numpy()

        y_batch_pred0 = y_batch_pred[j]
        pred0 = pred[j]
        mean = np.mean(x_batch0)
        std = np.std(x_batch0)
        y_batch_plot = y_batch_pred0 * std / 3.1 + mean
        y_pred_plot = pred0 * std / 2.9 + mean

        data_smooth = (
            pd.Series(x_batch0)
            .rolling(80, center=True, min_periods=1)
            .mean()
            .to_numpy()
        )
        plt.figure(figsize=(20, 10))
        plt.plot(data_smooth, label="data smooth")
        plt.plot(y_batch_plot, label="label true")
        plt.plot(y_pred_plot, label="label pred")
        plt.legend()
        plt.show()

    del y_pred, x_batch, y_batch, pred

print(f"Test Predictions: 0: {sum(test_preds==0)}, 1: {sum(test_preds==1)}")
print(f"Test Targets: 0: {sum(test_targets==0)}, 1: {sum(test_targets==1)}")

print("TEST_SCORE (F1): ", f1_score(test_targets, test_preds, average="macro"))
correct = (test_preds == test_targets).sum().item()

print(
    f"Accuracy of the network on the {test_shape[0]} train ranges: {100 * correct // total} %"
)